In [ ]:
import numpy as np
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, Slider, CustomJS, Text
from bokeh.plotting import Figure, show
from bokeh.io import output_notebook 



#### Testing two parameters (doesn't work yet) (maybe with pandas multilevel dataframes?)

#### Jaynes Cummings

In [ ]:
def c1t(t, lam = 1., R = .25, c10 = 1.):
    
    expt = lam * t / 2
    
    if R == .5:
        output = c10 * np.exp(-expt) * (1 + expt)
    #elif R == 0:
    #    output = c10 * np.exp(-expt) * (np.cosh(expt * sqt) + np.sinh(expt*sqt) / sqt)
    elif R < .5:
        sqt = np.sqrt(1-2*R)
        output = c10 * np.exp(-expt) * (np.cosh(expt * sqt) + np.sinh(expt*sqt) / sqt)
    elif R > .5:
        sqt = np.sqrt(-1+2*R)
        output = c10 * np.exp(-expt) * (np.cos(expt * sqt) + np.sin(expt*sqt) / sqt)

        
    return output

In [ ]:
ts = [t*0.02 for t in range(0, 500)]

Rmin = 0
Rmax = 10
Rstep = .2
Rrange = np.arange(Rmin, Rmax, Rstep)
Rrange_str = [str(i) for i in range(len(Rrange))]
lamrange = np.arange(.1,2.,.1)
lamrange_str = [str(lam) for lam in range(len(lamrange))]

Rrange_str_R = ['R = {:.1f}'.format(R) for R in Rrange]
#Rrange_str = ['{:.1f}'.format(i) for i in Rrange] # truncate to two decimals

# make a dictionary of form {'0': 0.0, '1': 0.2, .. }
Rrange_dict = {Rrange_str[i]:Rrange.round(2)[i] for i,_ in enumerate(Rrange)} # rounding to two decimals


ys = {r_str:{lam_str:[c1t(t, R = Rrange[int(r_str)], lam = lamrange[int(lam_str)])**2 for t in ts] for lam_str in lamrange_str} for r_str in Rrange_str}
#ys = {r_str:[c1t(t, R = Rrange_dict[r_str])**2 for t in ts] for r_str in Rrange_str}

initial_r = Rrange_str[1]
initial_lam = lamrange_str[1]


In [ ]:
rs = {Rrange_str[i] : [Rrange_str_R[i]] for i,_ in enumerate(Rrange)} 


In [ ]:
ys['0'].keys()

In [ ]:
# Wrap the data in two ColumnDataSources
source_visible = ColumnDataSource(data=dict(
    x = ts, y = ys[initial_r][initial_lam]))
source_available = ColumnDataSource(data=ys)

# Define plot elements
plot = Figure(plot_width=400, plot_height=400, x_range=(-.1, 10), y_range=(-.01, 1))
plot.line('x', 'y', source=source_visible, legend_label="ρ₁₁", line_width=3, line_alpha=0.6)
#plot.text(10,10,text='lalala',source=)

# Add text
text_source = ColumnDataSource({'r_value': ['%s' % Rrange_str_R[1]]})
r_available = ColumnDataSource(data=rs)
text = Text(x=7.5, y=.8, text='r_value', text_font_size='15pt')
plot.add_glyph(text_source, text)

# Add slider
slider = Slider(value=int(initial_r),
                start=np.min([int(i) for i in ys.keys()]),
                end=np.max([int(i) for i in ys.keys()]),
                step=1,
                show_value = False)

slider2 = Slider(value=int(initial_lam),
                start=np.min([int(i) for i in ys['0'].keys()]),
                end=np.max([int(i) for i in ys['0'].keys()]),
                step=1,
                show_value = False)



# Define CustomJS callback, which updates the plot based on selected function
# by updating the source_visible ColumnDataSource.
slider.callback = CustomJS(
    args=dict(source_visible=source_visible,
              source_available=source_available,
              text_source = text_source,
              r_available = r_available), code="""
        var r_idx = cb_obj.value;
        
        // Get the data from the data sources
        var data_visible = source_visible.data;
        var data_available = source_available.data;
        
        // Change y-axis data according to the selected value
        data_visible.y = data_available[r_idx];
        
        // text
        text_source.data = {'r_value': [String(r_available.data[r_idx])]};
        
        // Update the plot
        source_visible.change.emit();
    """)

slider2.callback = CustomJS(
    args=dict(source_visible=source_visible,
              source_available=source_available,
              text_source = text_source,
              r_available = r_available), code="""
        var lam_idx = cb_obj.value;
        
        // Get the data from the data sources
        var data_visible = source_visible.data;
        var data_available = source_available.data;
        
        // Change y-axis data according to the selected value
        data_visible.y = data_available[lam_idx];
        
        // Update the plot
        source_visible.change.emit();
    """)

layout = row(plot, slider)

output_notebook()

show(layout)